In [ ]:
import h5py
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import *
from tensorflow import keras
from tensorflow.keras import layers, models, Model
from sklearn.metrics import roc_curve, auc
import tensorflow.keras.backend as K
import qkeras
from qkeras import *

# max number of jet constituents
N = 32

with h5py.File('data.h5', 'r') as f:
    x = f['x'][:, :N, :]
    y = f['y'][:]

# min pt cut on jet constituents
x[x[:, :, 0] < 2] = 0

non_zero_counts = np.sum(np.any(x == 0, axis=(2)), axis=1)
plt.figure(figsize=(8, 6))
plt.hist(non_zero_counts, bins=range(0, N+2), edgecolor='black', alpha=0.5)
plt.xlabel(f"Number of zero-padded constituents per jet (max {N} constituents considered per jet)")
plt.ylabel("Count of jets")
plt.yscale("log")
plt.show()

# normalization of the pt feature by interquantile range
q5 = np.percentile(x[:, :, 0], 5)
q95 = np.percentile(x[:, :, 0], 95)
x[:, :, 0] = (x[:, :, 0] - 0) / (q95 - q5)

train_ratio = 0.3
val_ratio = 0.1
test_ratio = 1 - train_ratio - val_ratio
X_train_val, X_test, Y_train_val, Y_test = train_test_split(x, y, test_size = test_ratio, random_state = 42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size = val_ratio/(val_ratio + train_ratio), random_state = 42)
print('X_train shape: ' + str(X_train.shape))
print('X_val   shape: ' + str(X_val.shape))
print('X_test  shape: ' + str(X_test.shape))
print('Y_train shape: ' + str(Y_train.shape))
print('Y_val   shape: ' + str(Y_val.shape))
print('Y_test  shape: ' + str(Y_test.shape))
del X_train_val, Y_train_val

In [ ]:
phi_dim = 32
rho_dim = 32

def build_phi():
    input_constituent = layers.Input(shape=(N, 3), name='phi_input')
    x = QDense(phi_dim, activation='relu', use_bias=True, name='phi1',
               kernel_quantizer=quantized_bits(8, 0, alpha=1.0),
               bias_quantizer=quantized_bits(8, 0, alpha=1.0))(input_constituent)
    
    x = QDense(phi_dim, activation='relu', use_bias=True, name='phi2',
               kernel_quantizer=quantized_bits(8, 0, alpha=1.0),
               bias_quantizer=quantized_bits(8, 0, alpha=1.0))(x)
    
    x = QDense(phi_dim, activation='relu', use_bias=True, name='phi3',
               kernel_quantizer=quantized_bits(8, 0, alpha=1.0),
               bias_quantizer=quantized_bits(8, 0, alpha=1.0))(x)
    
    return models.Model(input_constituent, x, name="phi")

def build_phi_pointwiseConv1d():
    quantizer = quantized_bits(10, 0, alpha=1)
    quantized_relu = 'quantized_relu(10, 0)'
    # Conv1D treats this as a 1D sequence with a length of N, and 3 channels.
    input_constituent = layers.Input(shape=(N, 3), name='phi_input')
    #input_constituent = QActivation('quantized_relu(8, 0)')(input_constituent)
    
    x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi1')(input_constituent)
    x = QActivation(quantized_relu, name='relu1')(x)
    
    x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi2', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
    x = QActivation(quantized_relu, name='relu2')(x)
    
    x = QConv1D(filters=phi_dim, kernel_size=1, use_bias=True, name='phi3', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
    x = QActivation(quantized_relu, name='relu3')(x)
    
    return models.Model(input_constituent, x, name="phi")

def build_rho():
    quantizer = quantized_bits(10, 0, alpha=1)
    quantized_relu = 'quantized_relu(10, 0)'
    input_agg = layers.Input(shape=(phi_dim,), name='rho_input')
    
    x = QDense(rho_dim, use_bias=True, name='rho1', kernel_quantizer=quantizer, bias_quantizer=quantizer)(input_agg)
    x = QActivation(quantized_relu, name='relu4')(x)
    
    x = QDense(5, use_bias=True, name='rho2', kernel_quantizer=quantizer, bias_quantizer=quantizer)(x)
    x = QActivation('softmax', name='softmax')(x)
    
    return models.Model(input_agg, x, name="rho")

#phi = build_phi()
phi = build_phi_pointwiseConv1d()
rho = build_rho()

input_constituent = layers.Input(shape=(N, 3))
phi_output = phi(input_constituent)
agg = layers.GlobalMaxPooling1D()(phi_output)
rho_output = rho(agg)

model = models.Model(input_constituent, rho_output)

#phi.summary()
#rho.summary()

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.008),
              loss='categorical_crossentropy', metrics = ['accuracy'])
model.summary()

In [ ]:
tf.config.run_functions_eagerly(True)
history = model.fit(X_train, Y_train,
                    validation_data = (X_val, Y_val),
                    epochs=20, batch_size=256)

In [ ]:
plt.figure(figsize = (15,10))
axes = plt.subplot(2, 2, 1)
axes.plot(history.history['loss'], label = 'train loss')
axes.plot(history.history['val_loss'], label = 'val loss')
axes.legend(loc = "upper right")
axes.set_xlabel('Epoch')
axes.set_ylabel('Loss')

In [ ]:
Y_pred = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))))

In [ ]:
def plot_roc(y_test, y_pred, labels):
    for x, label in enumerate(labels):        
        fpr, tpr, _ = roc_curve(y_test[:, x], y_pred[:, x])
        plt.plot(fpr, tpr, label='{0} tagger, AUC = {1:.1f}'.format(label, auc(fpr, tpr)*100.), linestyle='-')
    #plt.semilogy()
    #plt.semilogx()
    plt.ylabel("Signal Efficiency")
    plt.xlabel("Background Efficiency")
    #plt.ylim(0.00001, 1)
    #plt.xlim(0.00001, 1)
    plt.grid(True)
    plt.legend(loc='best')  
    
plt.figure(figsize=(4, 4))
plot_roc(Y_test, Y_pred, ['g','q','w','z','t'])

In [ ]:
model.save('model.keras')

In [ ]:
model = tf.keras.models.load_model('model.keras')

In [ ]:
Y_pred = model.predict(X_test)

# HLS

In [ ]:
import hls4ml
import qkeras

In [ ]:
hls4ml.converters.convert_from_keras_model(model,
                                           output_dir='my-hls-test',
                                           project_name='myproject',
                                          )

In [ ]:
phi_layer = model.get_layer('time_distributed').layer
X_test_mask = layers.Masking(mask_value=0.)(X_test)
phi_outputs_mask = layers.TimeDistributed(phi_layer)(X_test_mask)
agg_mask = model.layers[2](phi_outputs_mask)
Y_pred_mask = model.layers[3](agg_mask)

In [ ]:
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred, axis=1))))

In [ ]:
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred_mask, axis=1))))

In [ ]:
Y_pred_mask

In [ ]:
Y_pred